In [1]:
!pip install imutils

  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25860 sha256=d59d278d66890534d46a9078bf109d2490b6ed6a7cb5065538728c42196d8692
  Stored in directory: /root/.cache/pip/wheels/86/d7/0a/4923351ed1cec5d5e24c1eaf8905567b02a0343b24aa873df2
Successfully built imutils


In [2]:
import tensorflow as tf 
import cv2 
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.python.keras.utils import generic_utils
from tensorflow.keras import models
from tensorflow.keras import layers

In [3]:
# img_size=224
# sample_img=cv2.imread("")
# print(sample_img.shape)
# plt.imshow(sample_img)
# resized_sample=cv2.resize(sample_img,(img_size,img_size))
# plt.imshow(resized_sampl)

In [4]:
DataDirectory="../input/face-emotions/Training Dataset"
Classes=["Happy","Sad","Scared","Surprised"]

training_Data= []

img_size=224

def Create_Training_Data():
    for category in Classes:
        path=os.path.join(DataDirectory,category)
        class_num=Classes.index(category)  #0,1,2....Labels
        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img))
                new_array=cv2.resize(img_array,(img_size,img_size))
                training_Data.append([new_array,class_num])
            except Exception as e:
                pass


Create_Training_Data()

import random
random.shuffle(training_Data)

X=[] #A list
y=[] #A list

for features,label in training_Data:
    X.append(features)
    y.append(label)

X=np.array(X).reshape(-1,img_size,img_size,3)  #Design Matrix size (shape= #no.of.images x pixels x pixels x 3)  
                                               #We put-1 so as to automatically detect the number of images

#Normalize the data
X=X/255.0 #Other way to normalize=Using scikit learn

Y=np.array(y) #Label Matrix (shape= #no.of.images x 1 )



In [5]:
print(X.shape)
print(Y.shape)
print(y[3])
print(y[4])
print(y[16])
print(y[232])
print(len(training_Data))
temp=np.array(training_Data)
temp.shape #This should be (no.of.images,2)

(6591, 224, 224, 3)
(6591,)
0
3
1
3
6591


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


(6591, 2)

In [6]:
model=tf.keras.applications.MobileNetV2() #Pretriained model
model.summary()

14540800/14536120 [==============================] - 0s 0us/step
Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
______________

In [9]:
#Remove/Modify some things in the huge model
base_input=model.layers[0].input   #'0' means that for input we are taking the first layer of the model
base_output=model.layers[-2].output #Discard the last layer i.e. -1 from the Pretrained model...start from -2

final_output=layers.Dense(128)(base_output)
final_output=layers.Activation('relu')(final_output)
final_output=layers.Dense(64)(final_output)
final_output=layers.Activation('relu')(final_output)
final_output= layers.Dense(4,activation='softmax')(final_output)  #3 classes

In [10]:
new_model=keras.Model(inputs=base_input,outputs=final_output)
new_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
____________________________________________________________________________________________

In [11]:
new_model.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics=["accuracy"])  #Settings for multi-class classification

In [12]:
new_model.fit(X,Y,epochs=2,validation_split=0.2)

Epoch 1/2
  7/165 [>.............................] - ETA: 10:44 - loss: 1.4745 - accuracy: 0.2996Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-e83c836e5049>", line 1, in <module>
    new_model.fit(X,Y,epochs=2,validation_split=0.2)
  File "/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py", line 1100, in fit
    tmp_logs = self.train_function(iterator)
  File "/opt/conda/lib/python3.7/site-packages/tensorflow/python/eager/def_function.py", line 828, in __call__
    result = self._call(*args, **kwds)
  File "/opt/conda/lib/python3.7/site-packages/tensorflow/python/eager/def_function.py", line 855, in _call
    return self._stateless_fn(*args, **kwds)  # pylint: disable=not-callable
  File "/opt/conda/lib/python3.7/site-packages/tensorflow/python/eager/function.py", line 2943, in __cal

TypeError: object of type 'NoneType' has no len()

In [ ]:
new_model.save('emotion_detector.h5')

In [ ]:
sample_test_img=cv2.imread("../input/covid-face-mask-detection-dataset/New Masks Dataset/Validation/Mask/1721.jpg")
plt.imshow(cv2.cvtColor(sample_test_img,cv2.COLOR_BGR2RGB)) #To display a single image just as an example
plt.show()

final_sample_test_img=cv2.resize(sample_test_img,(224,224))
final_sample_test_img=np.expand_dims(final_sample_test_img,axis=0) #We need a fourth dimension #This is similar to using -1
final_sample_test_img=final_sample_test_img/255.0

In [ ]:
Predictions=new_model.predict(final_sample_test_img)
Predictions

In [ ]:
#Testing the Model   trained_model.evaluate #For getting metrics on test data set